In [60]:
import pandas as pd
df = pd.read_csv("Data_Marketing_Customer_Analysis_Round3.csv")

In [61]:
# creating numerical_df
df = df.select_dtypes(include=['number'])

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,total_claim_amount
0,4809,48029,61,7,52,0,9,292
1,2228,92260,64,3,26,0,1,744
2,14947,22139,100,34,31,0,2,480
3,22332,49078,97,10,3,0,2,484
4,9025,23675,117,33,31,0,7,707
...,...,...,...,...,...,...,...,...
10684,15563,61541,253,12,40,0,7,1214
10685,5259,61146,65,7,68,0,6,273
10686,23893,39837,201,11,63,0,2,381
10687,11971,64195,158,0,27,4,6,618


### DATA WRANGLING

In [64]:
df = df.dropna()
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9082 entries, 0 to 10688
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   customer_lifetime_value        9082 non-null   int64
 1   income                         9082 non-null   int64
 2   monthly_premium_auto           9082 non-null   int64
 3   months_since_last_claim        9082 non-null   int64
 4   months_since_policy_inception  9082 non-null   int64
 5   number_of_open_complaints      9082 non-null   int64
 6   number_of_policies             9082 non-null   int64
 7   total_claim_amount             9082 non-null   int64
dtypes: int64(8)
memory usage: 638.6 KB


### SPLIT

In [42]:
 # Defining x explanatory variables and 1x y traget variable

In [43]:
X = df[["customer_lifetime_value","income","monthly_premium_auto","months_since_last_claim","months_since_policy_inception",
        "number_of_open_complaints","number_of_policies"]]
y = df[["total_claim_amount"]]

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 100)

### SCALING

In [46]:
# SCALE VARIABLES AFTER THE SPLIT (NOT TO MIX TEST AND TRAIN DATA)

In [47]:
import seaborn as sns

In [48]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
import numpy as np

In [66]:
from sklearn.preprocessing import PowerTransformer
import numpy as np

# Iniciated the PowerTransformer my my case
power_transformer = PowerTransformer()

# Fit the transformer to my X_train data 
X_train_transform = power_transformer.fit_transform(X_train)

# Transform X_test using the fitted transformer
X_test_fit = power_transformer.transform(X_test) 

# keeps them separately. It also ensures consistency in the transformation process between the two sets.

### Model training

In [67]:
from sklearn.linear_model import LinearRegression

In [68]:
# i train the model with the training sets from exploring var x and answers/labels y
lm = LinearRegression()
model = lm.fit(X_train_transform,y_train) 

In [69]:
model.coef_

array([[ -4.15820054, -32.32550692, 163.98060775,   4.99014802,
         -0.50043939,  -4.91536041,   0.4059141 ]])

In [70]:
model.intercept_

array([438.93716048])

### Model Interpretation

In [74]:
# -----  is this correct???? do i use the unscaled x_test? why? ----------
random_test = X_test.sample()

In [57]:
random_test

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
1452,11073,82807,92,15,20,0,2


In [19]:
model.predict(random_test)

/Users/bart/anaconda3/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


array([[-553098.71256027]])

### Create predictions for test set

In [20]:
y_pred = model.predict(X_test)

/Users/bart/anaconda3/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [32]:
y_pred = pd.DataFrame(y_pred)

In [33]:
y_test = y_test.reset_index(drop=True)

In [34]:
resiudals_df = pd.concat([y_test,y_pred],axis=1)

In [35]:
residuals_df = resiudals_df.rename(columns={"total_claim_amount":"y_test", 0:"y_pred"})

In [36]:
residuals_df["residual"] = residuals_df["y_test"]-residuals_df["y_pred"]

In [58]:
round(residuals_df,2)

,y_test,y_pred,residual
0,585,-764527.96,765112.96
1,601,-1034800.20,1035401.20
2,62,-2946894.31,2946956.31
3,18,-1921833.07,1921851.07
4,404,-939061.38,939465.38
...,...,...,...
2266,231,-1348473.73,1348704.73
2267,297,-1837627.68,1837924.68
2268,494,-761955.72,762449.72
2269,307,-968217.34,968524.34
